In [2]:
import random
import pandas as pd
import numpy as np

# Creation of the tile class
This class will be used to represent a cell in the grid and is responsible to hold information of the available options, if the cell has collapsed already and the value chosen iof applicable

In [19]:
class Tile:
    options = []
    collapsed = False
    value = None
    position = ()

    def __init__(self, x, y):
        self.options = [i for i in range(1, 10)] # set options for a cell from 1 to 9
        self.collapsed = False
        self.value = None
        self.position = (x, y)

    def setValue(self, v):
        self.value = v
        self.collapsed = True

    def __str__(self):
        return f'tile {self.position} \n' \
               f'options : {[i for i in self.options]} \n' \
               f'value : {self.value} \n' \
               f'collapsed : {self.collapsed} \n'


# Creation of the grid

In [25]:
grid = np.empty(shape=(9,9), dtype=Tile)
for x in range(9):
    for y in range(9):
        grid[x][y] = Tile(x, y)
